<img src="figures/logo.png" title="logo">

This is a preview chapter from my upcoming book, Mathematics of Machine Learning. The book is currently work in progress, but the early access version will be released soon. If you are interested in more, [sign up for updates](https://book.tivadardanka.com/) or [follow me on Twitter](https://twitter.com/TivadarDanka)!

<a id="pca"></a>
# PCA

The ability to look at the data with your eyes is one of the most under-appreciated things in machine learning. Before any machine learning is done, visualizing the data can reveal a lot of patterns. Exploring them can pay huge dividends later, as we can get a good intuition about what algorithm to use, which features to omit, etc.

However, there is a big issue: humans cannot see in dimensions larger than three. So, for real-life datasets, we have to perform certain tricks to benefit from the insight that visualization can provide. 

The Principal Component Analysis (PCA) is one of the most basic and useful methods for this purpose. PCA linearly transforms the data into a space that highlights the importance of each new feature, thus allowing us to prune the ones that don't reveal much. 

First, we will get an intuitive understanding of PCA, and then we proceed with a more detailed mathematical treatment.

## The idea behind PCA

To get a good grip on what PCA does, we shall see how it performs on a simple dataset, visualized below.

<img src="figures/01-data.png" style="height: 400px" title="A simple dataset">
<figcaption style="text-align: center;"><b>Figure 1.</b> A simple dataset.</figcaption>

Let $ X \in \mathbb{R}^{d \times n} $ denote our dataset. Each column encodes a sample, and each row represents a feature. For simplicity, we shall assume that $ d = 2 $ so we can visualize what is happening. However, all that will be said holds for the general case.

What we can observe right away is that the features $ x_1 $ and $ x_2 $ are perhaps not the best descriptors of our data. The distribution seems to be stretched out across both variables, although the shape suggests that the data can be simplified from a particular viewpoint. The features seem correlated: if $ x_1 $ is small, then $ x_2 $ is large. If $ x_1 $ is large, $ x_2 $ is small. This means that the variables contain redundant information. Redundancy is suboptimal when we have hundreds of features. If we could reshape the data to a form where the variables are uncorrelated and ordered according to importance, we would be able to discard those that are not that expressive.

We can formulate this process in the language of linear algebra. To decorrelate the data, we are looking to *diagonalize* the covariance matrix of $ X $. Since the covariance matrix is symmetric, the [spectral decomposition theorem](#spectral-decomposition-theorem) guarantees as an orthogonal matrix $ U $ such that

$$
    U^T \widehat{\mathrm{Cov}}[X] U = \Lambda = \mathrm{diag}(\lambda_1, \dots, \lambda_n).
$$

From the spectral theorem, we also know that we can select $ \Lambda $ such that the diagonal elements are in decreasing order. Since how the empirical covariance [behaves under linear transformations](#empirical-covariance), we can see that

$$
\begin{align*}
    U^T \widehat{\mathrm{Cov}}[X] U &= U^T \widehat{\mathrm{Cov}}[(X - \overline{X})]  U \\
    &= \widehat{\mathrm{Cov}}[U^T(X - \overline{X})].
\end{align*}
$$

If we define $ Y $ by

$$
    Y := U^T(X - \overline{X}),
$$

we can notice a few things right away. First, that $ X = UY + \overline{X} $, so $ X $ and $ Y $ are linear transformations of each other. Second, that $ \hat{\mathbb{E}}[Y] = 0 $, so the data is perfectly centered. However, most importantly, $ \widehat{\mathrm{Cov}}[Y] = \Lambda $ is diagonal. In other words, the features of $ Y $ are *uncorrelated*, which was our goal.

This is the essence of principal component analysis. Recall from the [spectral decomposition theorem](#spectral-decomposition-theorem) that the rows of $ U $ (denoted by $ \mathbf{u}_i $) form an orthonormal basis. $ Y $ is called the *principal component vector* of $ X $, while its features are called *principal components*.

Applied to our simple dataset, this is the result.

<img src="figures/02-data-pca.png" style="height: 400px" title="Principal components of X">
<figcaption style="text-align: center;"><b>Figure 2.</b> The principal component vectors of $ X $.</figcaption>

Note that since the components of $ \widehat{\mathrm{Cov}}[Y] $ are variances, they are all positive.

Each feature of $ Y $ is a linear combination of the features of $ X $. As we shall see next, this is not their only special property.

## The principal components maximize variance

Besides the uncorrelated nature $ Y $, one thing makes its features unique: each principal component is maximizing the variance of the data when projected on it.

<img src="figures/03-data-line-fit.png" style="height: 400px" title="The principal components maximize variance.">
<figcaption style="text-align: center;"><b>Figure 3.</b> The principal components maximize variance.</figcaption>

We can make this a bit more precise. For each unit vector $ \mathbf{v} $, we can define the transformation

$$
    \mathbf{v}^T (X - \overline{X}) \in \mathbb{R}^{1 \times n}.
$$

In case of $ \mathbf{v} = \mathbf{u}_i $, the result is the $ i $-th principal component vector! It turns out that $ \mathbf{u}_1 $ is the unit vector that maximizes the variance of this transformation. Indeed, we have

$$
\begin{align*}
    \widehat{\mathrm{Cov}}\Big[\mathbf{v}^T (X - \overline{X})\Big] &= \mathbf{v}^T \widehat{\mathrm{Cov}}\Big[ (X - \overline{X})\Big] \mathbf{v}.
\end{align*}
$$

Recall that we encountered this quantity [when we talked about the spectral decomposition](#spectral-decomposition-theorem). Since $ \mathbf{u}_1 $ is the first column of the diagonalizing orthogonal matrix,

$$
    \mathbf{u}_1 = \mathrm{arg} \max_{\| \mathbf{v} \| = 1} \mathbf{v}^T \widehat{\mathrm{Cov}}\Big[ (X - \overline{X})\Big] \mathbf{v}
$$

holds. In fact, even more is true: in general, the $ k $-th principal component $ \mathbf{u}_k $ is the unit vector that is orthogonal to $ \mathbf{u}_1, \dots, \mathbf{u}_{k-1} $ and maximizes the covariance of the projected data. Putting it into formulas,

$$
    \mathbf{u}_k = \mathrm{arg} \max_{\| \mathbf{v} \| = 1 \\ \mathbf{v} \perp \mathbf{u}_1, \dots, \mathbf{u}_{k-1}} \mathbf{v}^T \widehat{\mathrm{Cov}}\Big[ (X - \overline{X})\Big] \mathbf{v}
$$

holds.

In plain English, the $ k $-th principal component is selected such that

* it is uncorrelated with the previous ones, so it doesn't contain any redundant information,
* and it maximizes the available information (by maximizing variance).

## Dimensionality reduction with PCA

Now that we understand how PCA works, we should look into how it works on real datasets. Besides eliminating redundancies in the features, it can also be used to prune those that don’t convey much information. Recall that the covariance matrix of $ Y = U^T(X - \overline{X}) $ is diagonal, and the features' variance are ordered in decreasing order:

$$
    \widehat{\mathrm{Cov}}[Y] = \begin{bmatrix}
    \widehat{\mathrm{Cov}}[\mathbf{y}_1] & 0 & \dots & 0 \\
    0 & \widehat{\mathrm{Cov}}[\mathbf{y}_2] & \dots & 0 \\
    \vdots & \vdots & \ddots & \vdots \\
    0 & 0 & \dots & \widehat{\mathrm{Cov}}[\mathbf{y}_n]
    \end{bmatrix}, \quad \widehat{\mathrm{Cov}}[\mathbf{y}_1] \geq \widehat{\mathrm{Cov}}[\mathbf{y}_2] \geq \dots \widehat{\mathrm{Cov}}[\mathbf{y}_n].
$$

The *explained variance* of $ \mathbf{y}_k $ is defined by the ratio

$$
    \frac{\widehat{\mathrm{Cov}}[\mathbf{y}_k]}{\sum_{i=1}^{n} \widehat{\mathrm{Cov}}[\mathbf{y}_i] } \in [0, 1],
$$

which can be thought of as the percentage of "all the variance" in the data. Since the variances are decreasing, the most meaningful features come first. This means that if the cumulative explained variance

$$
    \sum_{j=1}^{k} \frac{\widehat{\mathrm{Cov}}[\mathbf{y}_j]}{\sum_{i=1}^{n} \widehat{\mathrm{Cov}}[\mathbf{y}_i] }
$$

is large enough for some $ k $, say around $ 90\% $, principal components after $ k $ can be dropped without significant information loss.

To see how it performs on real data, we will see how PCA performs on the famous [Iris dataset](https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html)! This dataset consists of four features (sepal length, sepal width, petal length, petal width) from three different kinds of irises: Setosa, Versicolour, and Virginica. This is how the dataset looks.

<img src="figures/04-iris-jointplot.png" style="height: 600px" title="The Iris dataset.">
<figcaption style="text-align: center;"><b>Figure 4.</b> The Iris dataset. The diagonal plots are density estimations of the features, the others are features plotted against each other.</figcaption>

Upon inspection, we can see that some combination of features separates the data well, some don’t. Their order definitely doesn’t signify their importance. By calculating the covariance matrix, we can also notice that they seem to be correlated:

```
[[ 0.68112222 -0.04215111  1.26582     0.51282889]
 [-0.04215111  0.18871289 -0.32745867 -0.12082844]
 [ 1.26582    -0.32745867  3.09550267  1.286972  ]
 [ 0.51282889 -0.12082844  1.286972    0.57713289]]
```

Let's apply PCA to see what it does with our dataset!

<img src="figures/05-iris-pca-jointplot.png" style="height: 600px" title="Principal components of the Iris dataset.">
<figcaption style="text-align: center;"><b>Figure 5.</b> Principal components of the Iris dataset.</figcaption>

The first two principal component almost completely describe the dataset, while the 3rd and 4th can be omitted without noticeable loss of information. The covariance matrix also looks much better. It is essentially diagonal, so the features are not correlated with each other.
```
[[ 1.57502004e+02 -3.33667355e-15  2.52281147e-15  5.42949940e-16]
 [-3.33667355e-15  9.03948536e+00  1.46458764e-15  1.37986097e-16]
 [ 2.52281147e-15  1.46458764e-15  2.91330388e+00  1.97218052e-17]
 [ 5.42949940e-16  1.37986097e-16  1.97218052e-17  8.87857213e-01]]
```

## What the PCA doesn't do

Although PCA is frequently used for feature engineering, there are limits on what it can do. For instance, if the dataset is thinly stretched out and the separating margin is small between them (like the example below), PCA won’t provide a representation where the difference between classes is sharper.

<img src="figures/06-pca-edge-case.png" style="height: 400px" title="A case when PCA doesn't really do much.">
<figcaption style="text-align: center;"><b>Figure 6.</b> A case when PCA doesn't really do much.</figcaption>

The reason is that the principal component vectors are given by an orthogonal transformation, and as we have mentioned, orthogonal transformations preserve distance. So, this doesn't stretch the space along any features. This is an advantage and a disadvantage at the same time.  In two dimensions, an orthogonal transformation is just a composition of rotations and reflections.

# Appendix: references

A few references in this preview chapter are required for complete understanding but are located elsewhere in the book. For the sake of completeness, I have included them here.

<a id="spectral-decomposition-theorem"></a>
## The spectral decomposition theorem

**Theorem.** *Let $ A \in \mathbb{R}^{n \times n} $ be a real symmetric matrix. (That is, $ A^T = A $.) Then $ A $ has exactly $ n $ real eigenvalues $ \lambda_1 \geq \dots \geq \lambda_n $, and the corresponding eigenvectors $ \mathbf{u}_1, \dots, \mathbf{u}_n $ can be selected such that they form an orthonormal basis.*

*Moreover, if we let $ \Lambda = \mathrm{diag}(\lambda_1, \dots, \lambda_n) $ and $ U $ be the orthogonal matrix whose rows are $ \mathbf{u}_1, \dots, \mathbf{u}_n $, then*

$$
    A = U \Lambda U^T
$$

*holds.*

The orthogonal matrix $ U $ and the corresponding orthonormal basis $ \{ \mathbf{u}_1, \dots, \mathbf{u}_n \} $ that diagonalizes a symmetric matrix $ A $ has a special property that is very important when discussing [principal component analysis](#PCA).

**Theorem.** *Let $ A \in \mathbb{R}^{n \times n} $ be a real symmetric matrix and let $ \lambda_1 \geq \dots \geq \lambda_n $ be its real eigenvalues in decreasing order. Moreover, let $ U \in \mathbb{R}^{n \times n} $ be the ortogonal matrix that diagonalizes $ A $, with the corresponding orthonormal basis $ \{ \mathbf{u}_1, \dots, \mathbf{u}_n \} $.*

*Then*

$$
    \mathrm{arg}\max_{\| \mathbf{x} \| = 1} \mathbf{x}^T A \mathbf{x} = \mathbf{u}_1,
$$

*and*

$$
    \max_{\| \mathbf{x} \| = 1} \mathbf{x}^T A \mathbf{x} = \lambda_1.
$$

**Proof.** Since $ \{ \mathbf{u}_1, \dots, \mathbf{u}_n \} $ is an orthonormal basis, any $ \mathbf{x} $ can be expressed as a linear combination of them:

$$
    \mathbf{x} = \sum_{i=1}^{n} x_i \mathbf{u}_i.
$$

Thus, since $ \mathbf{u}_i $ are the eigenvectors of $ A $,

$$
\begin{align*}
    Ax &= A \Big( \sum_{i=1}^{n} x_i \mathbf{u}_i \Big) = \sum_{i=1}^{n} x_i A \mathbf{u}_i = \sum_{i=1}^{n} x_i \lambda_i \mathbf{u}_i.
\end{align*}
$$

Plugging it back into $ x^T A x $, we have

$$
\begin{align*}
    x^T A x &= \Big( \sum_{j=1}^{n} x_j \mathbf{u}_j \Big)^T A \Big( \sum_{i=1}^{n} x_i \mathbf{u}_i \Big) \\
    &= \Big( \sum_{j=1}^{n} x_j \mathbf{u}_j^T \Big) A \Big( \sum_{i=1}^{n} x_i \mathbf{u}_i \Big) \\
    &= \Big( \sum_{j=1}^{n} x_j \mathbf{u}_j^T \Big) \Big( \sum_{i=1}^{n} x_i \lambda_i \mathbf{u}_i \Big) \\
    &= \sum_{i,j=1}^{n} x_i x_j \lambda_i \mathbf{u}_j^T \mathbf{u}_i.
\end{align*}
$$

Since the $ \mathbf{u}_i $-s form an orthonormal base,

$$
    \mathbf{u}_j^T \mathbf{u}_j = \begin{cases}
        1 \quad \text{if } i = j, \\
        0 \quad \text{otherwise.}
    \end{cases}
$$

In other words, $ \mathbf{u}_j^T \mathbf{u}_j $ vanishes when $ i \neq j $. Continuing the above calculation with this observation,

$$
\begin{align*}
    \mathbf{x}^T A \mathbf{x} &= \sum_{i,j=1}^{n} x_i x_j \lambda_i \mathbf{u}_j^T \mathbf{u}_i = \sum_{i=1}^{n} x_i^2 \lambda_i.
\end{align*}
$$

When $ \| \mathbf{x} \|^2 = \sum_{i=1}^{n} x_i^2 = 1 $, the sum $ \sum_{i=1}^{n} x_i^2 \lambda_i $ is a weighted average of the eigenvalues $ \lambda_i $. So,

$$
    \sum_{i=1}^{n} x_i^2 \lambda_i \leq \sum_{i=1}^{n} x_i^2 \max_{k = 1, \dots, n} \lambda_k = \max_{k = 1, \dots, n} \lambda_k = \lambda_1,
$$

from which $ \mathbf{x}^T A \mathbf{x} \leq \lambda_1 $ follows. (Recall that we can assume without loss in generality that the eigenvalues are decreasing.) On the other hand, by plugging in $ \mathbf{x} = \mathbf{u}_1 $, we can see that $ \mathbf{u}_1^T A \mathbf{u}_1 = \lambda_1 $, so the maximum is indeed attained. From these two, the theorem follows. $ \square $

The mathematical formalization of the above result doesn't reveal much. At this point, we don't have all the tools to see, but in connection to the principal component analysis, this says that the first principal vector is the one that maximizes variance.

Moreover, this was just a special case. The general theorem is as follows.

**Theorem.** *Let $ A \in \mathbb{R}^{n \times n} $ be a real symmetric matrix, let $ \lambda_1 \geq \dots \geq \lambda_n $ be its real eigenvalues in decreasing order. Moreover, let $ U \in \mathbb{R}^{n \times n} $ be the ortogonal matrix that diagonalizes $ A $, with the corresponding orthonormal basis $ \{ \mathbf{u}_1, \dots, \mathbf{u}_n \} $.*

*Then for all $ k = 1, \dots, n $, we have*

$$
    \mathrm{arg}\max_{\| \mathbf{x} \| = 1 \\ \mathbf{x} \perp \{ \mathbf{u}_1, \dots, \mathbf{u}_{k-1} \} } \mathbf{x}^T A \mathbf{x} = \mathbf{u}_k,
$$

*and*

$$
    \max_{\| \mathbf{x} \| = 1 \\ \mathbf{x} \perp \{ \mathbf{u}_1, \dots, \mathbf{u}_{k-1} \}} \mathbf{x}^T A \mathbf{x} = \lambda_k.
$$

**Proof.** The proof is almost identical to the previous one. Since $ \mathbf{x} $ is required to be orthogonal to $ \mathbf{u}_1, \dots, \mathbf{u}_{k-1} $, it can be expressed as

$$
    \mathbf{x} = \sum_{i=k}^{n} x_i \mathbf{u}_i.
$$

Following the calculations in the proof of the previous theorem, we have

$$
    \mathbf{x}^T A \mathbf{x} = \sum_{i=k}^{n} x_i^2 \lambda_i \leq \lambda_k.
$$

On the other hand, similarly as before, $ \mathbf{u}_k^T A \mathbf{u}_k = \lambda_k $, so the theorem follows. $ \square $

<a id="empirical-covariance"></a>
## Empirical covariance

The empirical mean of a data sample is defined by

$$
    \hat{\mathbb{E}}[X] := \frac{1}{m} \sum_{k=1}^{m} \mathbf{X}_k,
$$

While the empirical covariance is given by

$$
    \widehat{\mathrm{Cov}}[X, Y] := \frac{1}{m} \big(X - \overline{X}\big) \big(Y - \overline{Y}\big)^T.
$$

Like their counterparts, the empirical mean and covariance behave properly with respect to linear transformations. For any $ A, B \in \mathbb{R}^{n \times n} $, we have

$$
    \hat{\mathbb{E}}[AX] = A\hat{\mathbb{E}}[X],
$$

and

$$
    \widehat{\mathrm{Cov}}[AX, BY] = A \widehat{\mathrm{Cov}}[X, Y] B^T.
$$

$$
    \widehat{\mathrm{Cov}}[AX + m] = A \widehat{\mathrm{Cov}}[XA] A^T
$$